<h1> Unzip and Rename - NAble Policy Zipped Exports </h1>

### Import Modules, Create env Variables

In [ ]:
# data import and file manipulation
import os
import requests
from requests.structures import CaseInsensitiveDict
import json
import csv
import xlrd

# zip file manipulation
import zipfile

#data conditioning
import pandas as pd
import numpy as np
import re
import datetime as dt

#data visualization
[REDACTED]/.pyplot as plt
import seaborn as sns

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# dictionary Directory
dictionary_dir = 'd:/git/example_infrastructure_data_dev/dictionaries'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports'

In [ ]:
source_dir = 'D:/project_docs/abc_nable_migration/abc_nable_exports/scripts_and_automation/policy_exports/service_templates'

export_dir = 'D:/project_docs/abc_nable_migration/nable_conditioned_exports/scripts_and_automation'

## Pull list of all zip files and capture folder prefix name

In [ ]:
# pull all filenames walking through all folders (recursive going down the tree)

source_report = []
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if ('.zip' in file):
            info_dict = {}
            info_dict['filename'] = os.path.join(file)
            info_dict['fullPath'] = os.path.join(root,file)
            source_report.append(info_dict)

## Standardize Client Names

In [ ]:
df = pd.read_csv(f'{dictionary_dir}/client_name_standardization.dict')
client_rename_dict = {}
for index, row in df.iterrows():
    [REDACTED] = row['[REDACTED]']
    currentName = row['currentName']
    client_rename_dict[[REDACTED]] = currentName


def client_names(c_name):
    for k, v in client_rename_dict.items():
        try:
            result = re.sub(k.lower(), v, c_name.lower())
            if result != c_name:
                return v
                break
        except Exception as e:
            print(e)
            break
    return c_name


In [ ]:
df_zips = pd.DataFrame(source_report)

In [ ]:
df_zips

In [ ]:
for index,row in df_zips.iterrows():

    with zipfile.ZipFile(row['fullPath'], 'r') as zip:
        content_list = zip.namelist()
        result = re.findall(r'([^\.\/]+)\.xml',content_list[0])

        export_path = (f"{export_dir}/service_templates/Service Template - {result[0]}")
        try:
            os.makedirs(export_dir)
        except:
            pass
        print(f"Attempting to Unzip [{row['fullPath']}] to [{export_path}]")
        zip.extractall(export_path)

In [ ]:
service_xmls = []
for root, dirs, files in os.walk(export_dir):
    for file in files:
        if ('.xml' in file):
            info_dict = {}
            info_dict['templateName'] = re.findall(r'([^\/]+)\.xml',file)[0]
            info_dict['filename'] = os.path.join(file)
            info_dict['fullPath'] = os.path.join(root,file)
            service_xmls.append(info_dict)

In [ ]:
df_service_template_xmls = pd.DataFrame(service_xmls)

In [ ]:
def policy_name(string):
    result = (re.findall(r'\\Service\sTemplate\s\-\s([^\\]+)\\',string))
    return result[0]

In [ ]:
df_service_template_xmls['policyName'] = df_service_template_xmls['fullPath'].apply(policy_name)

In [ ]:
df_service_template_xmls = df_service_template_xmls[['policyName','templateName','filename','fullPath']]

In [ ]:
df_service_template_xmls

In [ ]:
df_service_template_xmls.to_csv('[REDACTED]/.csv',index=False)